In [1]:
from __future__ import division
font = {'size'   : 20}
import matplotlib
matplotlib.rc('font', **font)
import matplotlib.pylab as plt
import numpy as np

In [2]:
def getRows(dataBase,startInd,endInd):
    import sqlite3
    print(type(dataBase))
    input1 = dataBase+'.rdb'
    conn = sqlite3.connect(input1)
    c = conn.cursor()
    input2 = 'SELECT * FROM '+dataBase+'_ob'
    aa = c.execute(input2)
    rows = []
    count = 0
    for row in aa:
        if (count >= startInd) & (count < endInd):
            rows.append(row)
        count += 1
        if count == endInd:
            break
    return rows

In [3]:
def getStartTime(row):
    import datetime as dt
    start = dt.datetime(int(row[5][0:4]), int(row[5][5:7]), int(row[5][8:10]), int(row[5][11:13]), int(row[5][14:16]), int(float(row[5][17:-1])))
    return start

In [4]:
def getTimes(rows,startTime):
    import matplotlib.pyplot as plt
    import datetime as dt
    timeSec = [0]
    for afterInd in range(1,len(rows)):
        current = dt.datetime(int(rows[afterInd][5][0:4]), int(rows[afterInd][5][5:7]), int(rows[afterInd][5][8:10]), int(rows[afterInd][5][11:13]), int(rows[afterInd][5][14:16]), int(float(rows[afterInd][5][17:-1])))
        currentDiff = current - startTime
        currentDiffSeconds = currentDiff.days*(24*3600) + currentDiff.seconds
        timeSec.append(currentDiffSeconds)
    return timeSec

In [5]:
def extractTickData(rows,timeSec):
    import numpy as np
    buyPrices = []
    sellPrices = []
    buyTimes = []
    sellTimes = []
    buyQuantity = []
    buyLevels = []
    sellQuantity = []
    sellLevels = []
    for ii in range(len(rows)):
        if rows[ii][1] == 'buy':
            buyPrices.append(rows[ii][2])
            buyTimes.append(timeSec[ii])
            buyQuantity.append(rows[ii][3])
            buyLevels.append(rows[ii][4])
        else:
            sellPrices.append(rows[ii][2])
            sellTimes.append(timeSec[ii])
            sellQuantity.append(rows[ii][3])
            sellLevels.append(rows[ii][4])
    buyPrices = np.array(buyPrices)
    sellPrices = np.array(sellPrices)
    buyTimes = np.array(buyTimes)
    sellTimes = np.array(sellTimes)
    buyQuantity = np.array(buyQuantity)
    buyLevels = np.array(buyLevels)
    sellQuantity = np.array(sellQuantity)
    sellLevels = np.array(sellLevels)
    
    return buyPrices, sellPrices, buyTimes, sellTimes, buyQuantity, buyLevels, sellQuantity, sellLevels

In [6]:
def DHMS(seconds):
    import numpy as np
    days = np.floor(seconds/3600)
    hours = np.floor((seconds/3600) - days*24)
    minutes = np.floor(((seconds/3600) - days*24 - hours)*60)
    seconds = np.floor((((seconds/3600) - days*24 - hours)*60 - minutes)*60)
    return np.array([days,hours,minutes,seconds])

In [7]:
def plotTimes_momentum(sellTimes1,sellPrices1,buyTimes1,buyPrices1,label1,start):
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots()
    import datetime as dt
    ax.plot(buyTimes1,buyPrices1,'g-',label=label1+' Buy Prices')
    ax.plot(sellTimes1,sellPrices1,'r-',label=label1+' Sell Prices')
    plt.show()
    labels = []
    for ii in range(0,len(ax.get_xticklabels())):
        if ax.get_xticklabels()[ii].get_text() != '':
            timeInc = int(ax.get_xticklabels()[ii].get_text())
            dhms = DHMS(timeInc)
            step = dt.timedelta(days = dhms[0],hours = dhms[1],minutes = dhms[2],seconds = dhms[3]) 
            currentLabel = start + step
            labels.append(currentLabel.strftime('%Y:%m:%d:%H:%M:%S'))
        else:
            labels.append(ax.get_xticklabels()[ii].get_text())
    fig, ax = plt.subplots()
    ax.plot(buyTimes1,buyPrices1,'g-',label=label1+' Buy Prices')
    ax.plot(sellTimes1,sellPrices1,'r-',label=label1+' Sell Prices')
    ax.set_xticklabels(labels)
    plt.xlabel('Time')
    plt.ylabel(label1 + ' Buy and Sell Prices')
    plt.title(label1 + ' Bitcoin Bid/Ask')
    plt.legend()
    plt.show()

In [8]:
def timeFilterData(data,times,cutoff1,cutoff2):
    filteredData = data[(cutoff1 < times) & (times < cutoff2)]
    return filteredData

In [26]:
def formatTradeData_momentum(sellTimes1,sellPrices1,buyTimes1,buyPrices1,sellTimes1_ref,sellPrices1_ref,buyTimes1_ref,buyPrices1_ref,backwardHours):
    
    import numpy as np
    from scipy import interpolate
    from sklearn import linear_model
    from scipy.interpolate import interp1d
  
    leftPoint = max([buyTimes1[0],sellTimes1[0]])
    rightPoint = min([buyTimes1[-1],sellTimes1[-1]])
    trade_timeline = sellTimes1[(sellTimes1 >= leftPoint) & (sellTimes1 <= rightPoint)]
    
    f1 = interp1d(buyTimes1, buyPrices1)
    f4 = interp1d(sellTimes1, sellPrices1)
    buy_1_sync = f1(trade_timeline)
    sell_1_sync = f4(trade_timeline)
    
    leftPoint_ref = max([buyTimes1_ref[0],sellTimes1_ref[0]])
    rightPoint_ref = min([buyTimes1_ref[-1],sellTimes1_ref[-1]])
    time_ref = sellTimes1_ref[(sellTimes1_ref >= leftPoint_ref) & (sellTimes1_ref <= rightPoint_ref)]
    
    g1 = interp1d(buyTimes1_ref, buyPrices1_ref)
    g4 = interp1d(sellTimes1_ref, sellPrices1_ref)
    
    buy_1_ref_sync = g1(time_ref)
    sell_1_ref_sync = g4(time_ref)
    
    zScore1 = np.zeros(len(trade_timeline)) 
    zScore2 = np.zeros(len(trade_timeline)) 
    
  
    time0 = trade_timeline[0]
    count = 0
    for ii in range(0,len(zScore1)):
    
        currentTime = trade_timeline[ii]
        count = count + 1
    
        sellPrices1_past = sell_1_ref_sync[time_ref < currentTime]
        time_past = time_ref[time_ref < currentTime]

        time_filt = time_past - time_past[-1]

        sellPrices1_past2 = sellPrices1_past[time_filt > -3600*backwardHours]
        
        buyPrices1_past = buy_1_ref_sync[time_ref < currentTime]
        buyPrices1_past2 = buyPrices1_past[time_filt > -3600*backwardHours]

        standardDev_1 = np.std(sellPrices1_past2)
        meanHist_1 = np.mean(sellPrices1_past2)

        standardDev_2 = np.std(buyPrices1_past2)
        meanHist_2 = np.mean(buyPrices1_past2)
       
        zScore1[ii] = (sell_1_sync[ii]  - meanHist_1)/standardDev_1
        zScore2[ii] = (buy_1_sync[ii] - meanHist_2)/standardDev_2
        """
        if (abs(zScore1[ii]) > 10) | (abs(zScore2[ii]) > 10):
            print(len(buyPrices1_past2))
            print(currentTime)
            print(time_past[-10:-1])
            print(currentTime - 3600*backwardHours)
        """
            
    return zScore1, zScore2, trade_timeline, sell_1_sync, buy_1_sync,  

In [176]:
def momentum_trade(timeLine,sellPrices1,buyPrices1,zScore_1,zScore_2,trade_cutoff):
    import numpy as np
    bank1 = 10**4
    coins1 = 0

    bankTracker1 = []
    coinTracker1 = []

    zScoreTracker_1 = []
    zScoreTracker_2 = []
    cashOutTimes = []
    cashOut1 = []
    cashOutz1 = []
    cashOutz2 = []
    
    extremeFlag = 0
    betSize = 0.1
    
    for ii in range(0,len(timeLine)-1):
        
        currentTime = timeLine[ii]
        sellP1 = sellPrices1[ii]
        buyP1 = buyPrices1[ii]

        if zScore_1[ii] > trade_cutoff: # buy
            bank1 = bank1 - betSize*sellP1
            coins1 = coins1 + betSize
            extremeFlag = 1
        if zScore_2[ii] < -trade_cutoff: # sell
            bank1 = bank1 + betSize*buyP1
            coins1 = coins1 - betSize
            extremeFlag = 1
        
        if (abs(zScore_1[ii]) < .5) & (abs(zScore_2[ii]) < .5):
        
            if extremeFlag == 1:
                cashOutTimes.append(currentTime)
                if coins1 > 0:
                    bank1 = bank1 + coins1*buyP1
                    bank1 = bank1 - 0.002*coins1*buyP1
                else:
                    bank1 = bank1 + coins1*sellP1
                    bank1 = bank1 + 0.002*coins1*sellP1
         
                coins1 = 0
                extremeFlag = 0
                cashOut1.append(bank1)
                cashOutz1.append(zScore_1[ii])
                cashOutz2.append(zScore_2[ii])
                  
                
        bankTracker1.append(bank1)
        coinTracker1.append(coins1)
        
    currentTime = timeLine[-1]
    sellP1 = sellPrices1[-1]
    buyP1 = buyPrices1[-1]
    
    cashOutTimes.append(currentTime)
    if coins1 > 0:
        bank1 = bank1 + coins1*buyP1
        #bank1 = bank1 - 0.002*coins1*buyP1
    else:
        bank1 = bank1 + coins1*sellP1
        #bank1 = bank1 + 0.002*coins1*sellP1
    coins1 = 0
    bankTracker1.append(bank1)
    coinTracker1.append(coins1)
        
    bankTracker1 = np.array(bankTracker1)
    coinTracker1 = np.array(coinTracker1)

    cashOutTimes = np.array(cashOutTimes)
    cashOut1 = np.array(cashOut1)
    cashOutz1 = np.array(cashOutz1)
    cashOutz2 = np.array(cashOutz2)
    
    return bankTracker1, coinTracker1, cashOutTimes, cashOut1, cashOutz1, cashOutz2

In [11]:
def plotReturn_momentum(timeLine,bankTracker1,naiveWorthTracker1,startTime,cashOutTimes,cashOutBank,label1):
    import numpy as np
    fig, ax = plt.subplots()
    ax.plot(timeLine,bankTracker1[1:],'c-',label=u'Total Portfolio Pairs Trading')
    ax.plot(timeLine,10**4*np.ones(len(naiveWorthTracker1[1:])),'y-',label=u'Total Inactive Trading')
    ax.plot(cashOutTimes,cashOutBank,'g*',label=u'Pairs Trading Cash Out')
    plt.show()
    labels = []
    for ii in range(0,len(ax.get_xticklabels())):
        if ax.get_xticklabels()[ii].get_text() != '':
            timeInc = int(ax.get_xticklabels()[ii].get_text())
            dhms = DHMS(timeInc)
            step = dt.timedelta(days = dhms[0],hours = dhms[1],minutes = dhms[2],seconds = dhms[3]) 
            currentLabel = startTime + step
            labels.append(currentLabel.strftime('%Y:%m:%d:%H:%M:%S'))
        else:
            labels.append(ax.get_xticklabels()[ii].get_text())
    fig, ax = plt.subplots()
    ax.plot(timeLine,bankTracker1[1:],'c-',label=u'Total Portfolio Pairs Trading')
    ax.plot(timeLine,10**4*np.ones(len(naiveWorthTracker1[1:])),'y-',label=u'Total Inactive Trading')
    ax.plot(cashOutTimes,cashOutBank,'g*',label=u'Pairs Trading Cash Out')
    ax.set_xticklabels(labels)
    plt.xlabel('Time')
    plt.ylabel('Account Value')
    plt.title(label1 + ' Momentum Trading')
    plt.legend(loc = 0)
    plt.show()

In [12]:
rows1 = getRows('huobi',20*10**6,27*10**6)

<type 'str'>


In [13]:
rows1 = getRows('okcoin',0,4*10**6)

<type 'str'>


In [14]:
print(getStartTime(rows1[0]))

2014-01-01 21:59:52


In [15]:
startTime = getStartTime(rows1[0])
times1 = getTimes(rows1,startTime)
buyPrices1, sellPrices1, buyTimes1, sellTimes1, buyQuantity1, buyLevels1, sellQuantity1, sellLevels1 = extractTickData(rows1,times1)

In [16]:
plotTimes_momentum(sellTimes1,sellPrices1,buyTimes1,buyPrices1,'OK_coin',startTime)

In [237]:
import datetime as dt
startCutoff = dt.datetime(2014,1,2,10,0,0) - startTime
startCutOffSeconds = startCutoff.days*(24*3600) + startCutoff.seconds
trainCutoff = dt.datetime(2014,1,2,23,0,0) - startTime
trainCutoffSeconds = trainCutoff.days*(24*3600) + trainCutoff.seconds
endCutoff = dt.datetime(2014,1,4,2,0,0) - startTime
endCutoffSeconds = endCutoff.days*(24*3600) + endCutoff.seconds

In [255]:
import datetime as dt
startCutoff = dt.datetime(2014,1,6,0,0,0) - startTime
startCutOffSeconds = startCutoff.days*(24*3600) + startCutoff.seconds
trainCutoff = dt.datetime(2014,1,6,10,18,42) - startTime
trainCutoffSeconds = trainCutoff.days*(24*3600) + trainCutoff.seconds
endCutoff = dt.datetime(2014,1,7,9,50,14) - startTime
endCutoffSeconds = endCutoff.days*(24*3600) + endCutoff.seconds

In [265]:
import datetime as dt
startCutoff = dt.datetime(2014,1,7,0,0,0) - startTime
startCutOffSeconds = startCutoff.days*(24*3600) + startCutoff.seconds
trainCutoff = dt.datetime(2014,1,7,9,50,16) - startTime
trainCutoffSeconds = trainCutoff.days*(24*3600) + trainCutoff.seconds
endCutoff = dt.datetime(2014,1,8,0,51,33) - startTime
endCutoffSeconds = endCutoff.days*(24*3600) + endCutoff.seconds

In [266]:
buyPrices1_trade = timeFilterData(buyPrices1,buyTimes1,trainCutoffSeconds,endCutoffSeconds)
sellPrices1_trade = timeFilterData(sellPrices1,sellTimes1,trainCutoffSeconds,endCutoffSeconds)
buyTimes1_trade = timeFilterData(buyTimes1,buyTimes1,trainCutoffSeconds,endCutoffSeconds)
sellTimes1_trade = timeFilterData(sellTimes1,sellTimes1,trainCutoffSeconds,endCutoffSeconds)
buyQuantity1_trade = timeFilterData(buyQuantity1,buyTimes1,trainCutoffSeconds,endCutoffSeconds)
buyLevels1_trade = timeFilterData(buyLevels1,buyTimes1,trainCutoffSeconds,endCutoffSeconds)
sellQuantity1_trade = timeFilterData(sellQuantity1,sellTimes1,trainCutoffSeconds,endCutoffSeconds)
sellLevels1_trade = timeFilterData(sellLevels1,sellTimes1,trainCutoffSeconds,endCutoffSeconds)

In [19]:
import matplotlib.pyplot as plt
plt.plot(buyTimes1_trade,buyQuantity1_trade,'b-')
plt.xlabel('OK Coin sell times')
plt.ylabel('Ok Coin sell prices')
plt.title('Best Fit Parameters')
plt.show()

In [267]:
buyPrices1_trade = buyPrices1_trade[buyLevels1_trade == 1]
sellPrices1_trade = sellPrices1_trade[sellLevels1_trade == 1]
buyTimes1_trade = buyTimes1_trade[buyLevels1_trade == 1]
sellTimes1_trade = sellTimes1_trade[sellLevels1_trade == 1]

buyPrices1_ref = buyPrices1[buyLevels1 == 1]
sellPrices1_ref = sellPrices1[sellLevels1 == 1]
buyTimes1_ref = buyTimes1[buyLevels1 == 1]
sellTimes1_ref = sellTimes1[sellLevels1 == 1]

In [21]:
plt.plot(buyPrices1_trade)
plt.show()

In [268]:
backwardHours = 6
zScore1,zScore2,timeLine,sellPrices1_trade_sync,buyPrices1_trade_sync = formatTradeData_momentum(sellTimes1_trade,sellPrices1_trade,buyTimes1_trade,buyPrices1_trade,sellTimes1_ref,sellPrices1_ref,buyTimes1_ref,buyPrices1_ref,backwardHours)

In [193]:
np.sign(0)

0

In [276]:
def momentumDir(zScore1,zScore2,trade_cutoff,sellPrices,buyPrices):
    momentumDirBuy = np.zeros(len(zScore1)-1)
    momentumDirSell = np.zeros(len(zScore1)-1)
    truthDirBuy = np.zeros(len(zScore1)-1)
    truthDirSell = np.zeros(len(zScore1)-1)
    posTradingDiffs = []
    negTradingDiffs = []
    for ii in range(0,len(momentumDirBuy)):
        if zScore1[ii] > trade_cutoff:
            momentumDirSell[ii] = 1
            posTradingDiffs.append(sellPrices[ii+1]-sellPrices[ii])
        if zScore1[ii] < -trade_cutoff:
            momentumDirSell[ii] = -1
            negTradingDiffs.append(sellPrices[ii+1]-sellPrices[ii])
        if zScore2[ii] > trade_cutoff:
            momentumDirBuy[ii] = 1
            posTradingDiffs.append(buyPrices[ii+1]-buyPrices[ii])
        if zScore2[ii] < -trade_cutoff:
            momentumDirBuy[ii] = -1
            negTradingDiffs.append(buyPrices[ii+1]-buyPrices[ii])
        truthDirBuy[ii] = np.sign(buyPrices[ii+1]-buyPrices[ii])
        truthDirSell[ii] = np.sign(sellPrices[ii+1]-sellPrices[ii])
    return momentumDirBuy,momentumDirSell,truthDirBuy,truthDirSell,posTradingDiffs,negTradingDiffs

In [279]:
trade_cutoff = 0
bankTracker1, coinTracker1, cashOutTimes,cashOut1, cashOutz1, cashOutz2 = momentum_trade(timeLine,sellPrices1_trade_sync,buyPrices1_trade_sync,zScore1,zScore2,trade_cutoff)

In [280]:
profit = bankTracker1[-1] - 10**4
print "Profit: $", profit

Profit: $ -56129.6252984


In [289]:
momentumDirBuy,momentumDirSell,truthDirBuy,truthDirSell,posTradingDiffs,negTradingDiffs = momentumDir(zScore1,zScore2,trade_cutoff,sellPrices1_trade_sync,buyPrices1_trade_sync)

In [282]:
print "Momentum Buy accuracy: ", sum(momentumDirBuy == truthDirBuy)/len(truthDirBuy)
print "Momentum Sell accuracy: ", sum(momentumDirSell == truthDirSell)/len(momentumDirSell)

Momentum Buy accuracy:  0.149136309218
Momentum Sell accuracy:  0.168576477966


In [283]:
print "correct: ",(np.sum(np.array(posTradingDiffs) > 0)+np.sum(np.array(negTradingDiffs) < 0))/(len(posTradingDiffs)+len(negTradingDiffs))
print "wrong: ",(np.sum(np.array(posTradingDiffs) < 0)+np.sum(np.array(negTradingDiffs) > 0))/(len(posTradingDiffs)+len(negTradingDiffs))
print "zeros: ",(np.sum(np.array(posTradingDiffs) == 0)+np.sum(np.array(negTradingDiffs) == 0))/(len(posTradingDiffs)+len(negTradingDiffs))

correct:  0.158856393592
wrong:  0.157032096232
zeros:  0.684111510176


In [294]:
np.round(.501)

1.0

In [287]:
nTrade = len(buyPrices1_trade_sync)-1

In [290]:
profit0,a0,b0 = simple_trade(buyPrices1_trade_sync[0:nTrade],sellPrices1_trade_sync[0:nTrade],buyPrices1_trade_sync[-1],sellPrices1_trade_sync[-1],momentumDirBuy,momentumDirSell)
print "Profit: $", profit0

Profit: $ 100650.594999


In [37]:
fig, ax = plt.subplots()
xaxis = timeLine
ax.plot(timeLine,bankTracker1,'r',label='momentum trade')
ax.plot(timeLine,10000*np.ones(len(bankTracker1)),'b',label='do nothing')
plt.show()
labels = []
for ii in range(0,len(ax.get_xticklabels())):
    if ax.get_xticklabels()[ii].get_text() != '':
        timeInc = int(ax.get_xticklabels()[ii].get_text())
        dhms = DHMS(timeInc)
        step = dt.timedelta(days = dhms[0],hours = dhms[1],minutes = dhms[2],seconds = dhms[3]) 
        currentLabel = startTime + step
        labels.append(currentLabel.strftime('%m/%d %H:%M'))
    else:
        labels.append(ax.get_xticklabels()[ii].get_text())
fig, ax = plt.subplots()
ax.plot(timeLine,bankTracker1,'r',label='momentum trade')
ax.plot(timeLine,10000*np.ones(len(bankTracker1)),'b',label='do nothing')
plt.legend(loc=(0,.8))
plt.ylabel("price")
plt.xlabel("time")
plt.title("BTC Account Value with Pairs Trading")
ax.set_xticklabels(labels)
plt.show()

In [69]:
plotReturn_momentum(timeLine,bankTracker1,naiveWorthTracker1,startTime,cashOutTimes,cashOut1,'OK Coin')

In [28]:
bankTracker1[-1]/10000

1.1554252123979114

In [71]:
fig, ax = plt.subplots()
ax.plot(timeLine,zScore2,'b',label=u'Ok Coin')
ax.plot(cashOutTimes,cashOutz2,'wo',label=u'OK Coin cash out')
plt.show()
labels = []
for ii in range(0,len(ax.get_xticklabels())):
    if ax.get_xticklabels()[ii].get_text() != '':
        timeInc = int(ax.get_xticklabels()[ii].get_text())
        dhms = DHMS(timeInc)
        step = dt.timedelta(days = dhms[0],hours = dhms[1],minutes = dhms[2],seconds = dhms[3]) 
        currentLabel = startTime + step
        labels.append(currentLabel.strftime('%Y:%m:%d:%H:%M:%S'))
    else:
        labels.append(ax.get_xticklabels()[ii].get_text())
fig, ax = plt.subplots()
ax.plot(timeLine,zScore2,'b',label=u'Ok Coin')
ax.plot(cashOutTimes,cashOutz2,'wo',label=u'Ok Coin cash out')   
ax.set_xticklabels(labels)
plt.xlabel('Time')
plt.ylabel('Z Score')
plt.title('Huobi and Ok Coin Z scores with Cashout times (z_threshold = 2)')
plt.legend(loc = 0)
plt.show()

In [72]:
fig, ax = plt.subplots()
ax.plot(timeLine,coinTracker1[1:],'r',label=u'btce')
ax.plot(timeLine,coinTracker2[1:],'b',label=u'bitstamp')
plt.show()
labels = []
for ii in range(0,len(ax.get_xticklabels())):
    if ax.get_xticklabels()[ii].get_text() != '':
        timeInc = int(ax.get_xticklabels()[ii].get_text())
        dhms = DHMS(timeInc)
        step = dt.timedelta(days = dhms[0],hours = dhms[1],minutes = dhms[2],seconds = dhms[3]) 
        currentLabel = startTime + step
        labels.append(currentLabel.strftime('%Y:%m:%d:%H:%M:%S'))
    else:
        labels.append(ax.get_xticklabels()[ii].get_text())
fig, ax = plt.subplots()
ax.plot(timeLine,coinTracker1[1:],'r',label=u'btce')
ax.plot(timeLine,coinTracker2[1:],'b',label=u'bitstamp') 
ax.set_xticklabels(labels)
plt.xlabel('Time')
plt.ylabel('Bitcoins')
plt.title('BTCE and Bitstamp Bitcoins with Cashout times (z_threshold = 2)')
plt.legend(loc = 0)
plt.show()

In [73]:
fig, ax = plt.subplots()
ax.plot(timeLine,bankTracker1[1:],'r',label=u'btce')
ax.plot(timeLine,bankTracker2[1:],'b',label=u'bitstamp')
ax.plot(cashOutTimes,cashOut1,'yo',label=u'btce cash out')
ax.plot(cashOutTimes,cashOut2,'wo',label=u'bitstamp cash out')
plt.show()
labels = []
for ii in range(0,len(ax.get_xticklabels())):
    if ax.get_xticklabels()[ii].get_text() != '':
        timeInc = int(ax.get_xticklabels()[ii].get_text())
        dhms = DHMS(timeInc)
        step = dt.timedelta(days = dhms[0],hours = dhms[1],minutes = dhms[2],seconds = dhms[3]) 
        currentLabel = startTime + step
        labels.append(currentLabel.strftime('%Y:%m:%d:%H:%M:%S'))
    else:
        labels.append(ax.get_xticklabels()[ii].get_text())
fig, ax = plt.subplots()
ax.plot(timeLine,bankTracker1[1:],'r',label=u'btce')
ax.plot(timeLine,bankTracker2[1:],'b',label=u'bitstamp')
ax.plot(cashOutTimes,cashOut1,'yo',label=u'btce cash out')
ax.plot(cashOutTimes,cashOut2,'wo',label=u'bitstamp cash out')   
ax.set_xticklabels(labels)
plt.xlabel('Time')
plt.ylabel('Account Values')
plt.title('BTCE and Bitstamp Account Values with Cashout times (z_threshold = 2)')
plt.legend(loc = 0)
plt.show()

In [31]:
100*(((bankTracker1[-1]+bankTracker2[-1])/20000)-1)

564.97686091810044

In [82]:
import numpy as np
backwardHours = 6
trade_cutoff_vector = np.linspace(.6,4,40)
tradeReturn = np.zeros(len(trade_cutoff_vector))
zScore1,zScore2,timeLine,sellPrices1_trade_sync,buyPrices1_trade_sync,sellPrices2_trade_sync,buyPrices2_trade_sync = formatTradeData(sellTimes1_trade,sellPrices1_trade,buyTimes1_trade,buyPrices1_trade,sellTimes2_trade,sellPrices2_trade,buyTimes2_trade,buyPrices2_trade,sellTimes1_ref,sellPrices1_ref,buyTimes1_ref,buyPrices1_ref,sellTimes2_ref,sellPrices2_ref,buyTimes2_ref,buyPrices2_ref,backwardHours,slopeToUse_1,interceptToUse_1,slopeToUse_2,interceptToUse_2)
for ii in range(0,len(trade_cutoff_vector)): 
    trade_cutoff = trade_cutoff_vector[ii]
    bankTracker1, naiveWorthTracker1, bankTracker2, naiveWorthTracker2, coinTracker1, coinTracker2, cashOutTimes,cashOut1, cashOut2, cashOutz1, cashOutz2 = pairs_trade(timeLine,sellPrices1_trade_sync,buyPrices1_trade_sync,sellPrices2_trade_sync,buyPrices2_trade_sync,zScore1,zScore2,trade_cutoff)
    tradeReturn[ii] = 100*(((bankTracker1[-1]+bankTracker2[-1])/20000)-1)
plt.plot(trade_cutoff_vector,tradeReturn,'b')
plt.xlabel('Z Score Threshold')
plt.ylabel('Percent Return')
plt.title('Pairs Trading Return')
plt.show()

In [55]:
import numpy as np
backwardHoursVec = [1/12,1/2,1,2,3,4,5,6,7,8,9,10,11,12]
trade_cutoff_vector = np.linspace(.6,4,20)
tradeReturn = np.zeros((14,len(trade_cutoff_vector)))
for ii in range(0,14):
    backwardHours = backwardHoursVec[ii]
    zScore1,zScore2,timeLine,sellPrices1_trade_sync,buyPrices1_trade_sync,sellPrices2_trade_sync,buyPrices2_trade_sync = formatTradeData(sellTimes1_trade,sellPrices1_trade,buyTimes1_trade,buyPrices1_trade,sellTimes2_trade,sellPrices2_trade,buyTimes2_trade,buyPrices2_trade,sellTimes1_ref,sellPrices1_ref,buyTimes1_ref,buyPrices1_ref,sellTimes2_ref,sellPrices2_ref,buyTimes2_ref,buyPrices2_ref,backwardHours,slopeToUse_1,interceptToUse_1,slopeToUse_2,interceptToUse_2)
    for jj in range(len(trade_cutoff_vector)):
        trade_cutoff = trade_cutoff_vector[jj]
        bankTracker1, naiveWorthTracker1, bankTracker2, naiveWorthTracker2, coinTracker1, coinTracker2, cashOutTimes,cashOut1, cashOut2, cashOutz1, cashOutz2 = pairs_trade(timeLine,sellPrices1_trade_sync,buyPrices1_trade_sync,sellPrices2_trade_sync,buyPrices2_trade_sync,zScore1,zScore2,trade_cutoff)
        tradeReturn[ii,jj] = 100*(((bankTracker1[-1]+bankTracker2[-1])/20000)-1)

In [57]:
fig, ax = plt.subplots()
ax.imshow(tradeReturn)
xlabels = [item.get_text() for item in ax.get_xticklabels()]
ylabels = [item.get_text() for item in ax.get_yticklabels()]
ax.set_xticklabels(['','0.6','1.6','2.6','3.6',''])
ax.set_yticklabels(['','1/12','1','2','4','6','8','10','12',''])
plt.xlabel('Z-score threshold')
plt.ylabel('Backward Window (hours)')
plt.title('BTCE and Bitstamp Pairs Trading Return')
plt.colorbar
plt.show()

In [75]:
def simple_trade(buyPrice,sellPrice,lastBuyPrice,lastSellPrice,predictionDirB,predictionDirS):
    bank1 = 10**3
    assets1 = 0

    bankTracker1 = [bank1]
    assetTracker1 = [0]

    betSize = .1
    
    for ii in range(0,len(buyPrice)):
    
        buyP1 = buyPrice[ii]
        sellP1 = sellPrice[ii]
        
        if predictionDirS[ii] > 0: # buy
            bank1 = bank1 - betSize*sellP1
            assets1 = assets1 + betSize
        if predictionDirB[ii] < 0: # sell
            bank1 = bank1 + betSize*buyP1
            assets1 = assets1 - betSize
            
        bankTracker1.append(bank1)
        assetTracker1.append(assets1)
        
    # close position at end
    if assets1 > 0: # sell
        bank1 = bank1 + assets1*lastBuyPrice
        assets1 = 0
    if assets1 < 0: # buy
        bank1 = bank1 + assets1*lastSellPrice
        assets1 = 0
        
    bankTracker1.append(bank1)
    assetTracker1.append(assets1)

    bankTracker1 = np.array(bankTracker1)
    assetTracker1 = np.array(assetTracker1)
    profit = bankTracker1[-1] - bankTracker1[0]
    
    return profit, bankTracker1, assetTracker1